In [137]:
class Nodo():
    def __init__(self,_estado,_padre,_meta,_costo_g):
        self.estado=_estado   #Entendemos por estado (fila,columna)
        self.padre=_padre
        self.meta=_meta
        #self.inicio=_inicio

        filaA,colA=self.estado
        filaM,colM=self.meta

        #filaI,colI=self.inicio

        self.distancia_greedy=(abs(colM-colA)+abs(filaM-filaA))
        #self.distancia_star=((abs(colI-colA)+abs(filaI-filaA)) + (abs(colM-colA)+abs(filaM-filaA)))
        self.costo_g = _costo_g
        self.distancia_star=self.costo_g + self.distancia_greedy

In [138]:
class FronteraStack():
    def __init__(self):
        self.frontera=[]
    def __str__(self):
        return "Nodos en la frontera: " + " ".join(
            str(nodo.estado) for nodo in self.frontera)
    def agregar_nodo(self,_nodo):
        #Agregar el nodo pasado por parametro a la frontera
        self.frontera.append(_nodo)
    def quitar_nodo(self):
        #Quitar nodo de la frontera (respetar el tipo de frontera)
        return self.frontera.pop()
    def esta_vacia(self):
        #Comprobar si la frontera está vacia o no
        return len(self.frontera)==0
    def contiene_estado(self,_estado):
        #Comprobar si el estado pasado por parametro ya se encuentra en la frontera
        for nodo in self.frontera:
            if nodo.estado == _estado:
                return True
        return False

In [139]:
class FronteraQueue(FronteraStack):
    '''Aplicar herencia con FronteraStack
       La unica diferencia entre ambas es como
       se quitan los nodos
    '''
    def quitar_nodo(self):
        return self.frontera.pop(0)

In [140]:
'''class FronteraGreedy(FronteraStack):
  def quitar_nodo(self):
    menor=999
    for nodo in self.frontera:
     # print(nodo.distancia)
      if nodo.distancia_greedy<menor:
        menor=nodo.distancia_greedy
    for nodo in self.frontera:
      if(nodo.distancia_greedy==menor):
        self.frontera.remove(nodo)
        return nodo'''

'class FronteraGreedy(FronteraStack):\n  def quitar_nodo(self):\n    menor=999\n    for nodo in self.frontera:\n     # print(nodo.distancia)\n      if nodo.distancia_greedy<menor:\n        menor=nodo.distancia_greedy\n    for nodo in self.frontera:\n      if(nodo.distancia_greedy==menor):\n        self.frontera.remove(nodo)\n        return nodo'

In [141]:
class FronteraGreedy(FronteraStack):
  def quitar_nodo(self):
    self.frontera.sort(key=lambda n: (n.distancia_greedy))

    return self.frontera.pop(0)

In [142]:
class FronteraGreedy_Star(FronteraStack):
  def quitar_nodo(self):
    self.frontera.sort(key=lambda n: (n.distancia_star))

    return self.frontera.pop(0)

In [143]:
class Laberinto():
    def  __init__(self,_algoritmo):
        '''Dentro del init podemos ejecutar funciones
           para ir definiendo los atributos de la clase.
           Les dejo lista la parte de leer el laberinto
           del archivo de texto, y la detección del inicio,
           meta y paredes.
        '''
          
        
        with open('C:/Users/Jordy/Downloads/Laberinto_4','r') as archivo:
            laberinto=archivo.read()     #Con read() leemos todo el archivo y lo guardamos en laberinto
        self.laberinto=laberinto.splitlines() #Con splitlines() separamos el laberinto en lineas, eliminando el \n
        self.ancho=len(self.laberinto[0])    #El ancho del laberinto es la cantidad
                                        #de caracteres de la primer linea
                                        #(O de cualquiera suponiendo que todas tienen el mismo ancho)
        self.alto=len(self.laberinto)        #El alto del laberinto es la cantidad de lineas
        self.paredes=[]                 #Lista de paredes

        for fila in range(self.alto):   #Recorremos todas las filas

            fila_paredes=[]             #Creamos una lista vacia para las paredes de la fila actual
                                        #para cada fila se vuelve a limpiar la lista
            for columna in range(self.ancho): #Recorremos todas las columnas
                if self.laberinto[fila][columna]==' ': #Si el caracter es # es una pared
                    fila_paredes.append(False) #Agregamos la pared a la lista de paredes de la fila actual
                    
                elif self.laberinto[fila][columna]=='I':   #Si el caracter es I es el inicio
                    self.inicio=(fila,columna)

                    fila_paredes.append(False)         #Guardamos el inicio
                elif self.laberinto[fila][columna]=='M':   #Si el caracter es M es la meta
                    self.meta=(fila,columna)
                    fila_paredes.append(False)           #Guardamos la meta
                else:
                    fila_paredes.append(True)
                    
            self.paredes.append(fila_paredes)         #Agregamos la lista de paredes de la fila actual a la lista de paredes

        #De este modo ya tenemos identificadas las paredes, el inicio y la meta
        self.solucion=None
        self.algoritmo=_algoritmo #String en el que pasamos el nombre del algoritmo a utilizar
    
    
    
    def expandir_nodo(self,_nodo):
        '''Dentro de _nodo.estado tenemos la posicion actual del nodo
           Debemos comprobar en todas las direcciones si podemos movernos
           descartando las que sean paredes o esten fuera del laberinto                 (i-1,j)
           Utilicen el grafico que está en el Notion para guiarse                (i,j-1) (i,j) (i,j+1)
           Devolver una lista de vecinos posibles (nodos hijo)                          (i+1,j)
        '''
        #        (0,2)
        #   (1,2)(1,2)(1,3)
        #        (2,2)
        # Nodo estado:(fila,columna) padre:...
        fila, columna = _nodo.estado

        #                    (2,2)          (1,2)           (0,2)           (1,3)
        posiciones = [(fila+1,columna),(fila,columna-1),(fila-1,columna),(fila,columna+1)]

        vecinos = []
        for f,c in posiciones:
            if 0 <= f < self.alto and 0 <= c < self.ancho and not self.paredes[f][c]:
                vecinos.append((f,c))


        return vecinos

    """def heuristica(self,_nodo):
      fila,columna = _nodo.estado

      distancia=(abs(fila-self.meta[0]) + abs(columna-self.meta[1]))

      return distancia"""
    def dibujar(self,solucion):
         #######INICIO DEL CODIGO PARA GRAFICAR########
        import pygame

        # Inicializar Pygame
        pygame.init()

        # Definir el tamaño de la ventana
        width = 600
        height = 400
        window = pygame.display.set_mode((width, height))

        # Definir los colores
        black = (0, 0, 0)
        white = (255, 255, 255)
        red=(255,0,0)
        gold=(255,215,0)
        green=(0,128,0)
        darkgrey=(169,169,169)
        greenyellow=(173,255,47)
        hotpink=(255,105,180)
        lightcoral=(240,128,128)
        maroon=(128,0,0)
        orange=(255,165,0)
        orangered=(255,69,0)

        # Definir el tamaño de los bloques del laberinto
        block_size = 20
        
        # Establece el título de la ventana
        pygame.display.set_caption("|L|A|B|E|R|I|N|T|O")
        
        # Ciclo principal del juego
        done = False
        while not done:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    done = True

        # Limpia la pantalla
            window.fill(black)

        # Dibuja el laberinto
            for fila in range(self.alto):
                for col in range(self.ancho):
                    if self.laberinto[fila][col] =='#':
                        pygame.draw.rect(window, orange,
                                         (col * block_size,
                                          fila * block_size,
                                          block_size,
                                          block_size))
                    elif self.laberinto[fila][col]==' ':
                        pygame.draw.rect(window, darkgrey,
                                         (col * block_size,
                                          fila * block_size,
                                          block_size,
                                          block_size))
                    elif self.laberinto[fila][col]=='I':
                        pygame.draw.rect(window, red,
                                         (col * block_size,
                                          fila * block_size,
                                          block_size,
                                          block_size))
                    elif self.laberinto[fila][col]=='M':
                        pygame.draw.rect(window, maroon,
                                         (col * block_size,
                                          fila * block_size,
                                          block_size,
                                          block_size))

        # Dibuja la solución del laberinto
            for i in range(len(solucion)-1):
                pygame.draw.line(window, greenyellow,
                                 [solucion[i][1]*20+10, solucion[i][0]*20+10],
                                 [solucion[i+1][1]*20+10, solucion[i+1][0]*20+10], 5)

            # Actualiza la pantalla
            pygame.display.update()

        # Cierra Pygame
        pygame.quit()
        
        
    
    def resolver(self):
        '''
        Acá tienen que implementar el algoritmo de busqueda
        La idea es intentar replicar el pseudocodigo que vimos en clase
        1- Inicializar la frontera con el nodo inicial
        2- Inicializar el conjunto de explorados como vacio
        3- Repetimos:
            3.1- Si la frontera esta vacia, no hay solucion
            3.2- Quitamos un nodo de la frontera
            3.3- Si el nodo contiene un estado que es meta, devolver la solucion
            3.4- Agregar el nodo a explorados
            3.5- Expandir el nodo, agregando los nodos hijos a la frontera
        '''
        if self.algoritmo=='BFS':
            #Crear la frontera que corresponda
            frontera = FronteraQueue()
        elif self.algoritmo=='DFS':
            #Crear la frontera que corresponda
            frontera = FronteraStack()
        elif self.algoritmo=='GBS':
            frontera = FronteraGreedy()
        elif self.algoritmo=='GS':
            frontera = FronteraGreedy_Star()
        #------------------------------------------------------------------------
        #self.costo=0
        nodo_inicial = Nodo(self.inicio,None,self.meta,0)
        frontera.agregar_nodo(nodo_inicial)
        self.explorados = []
        self.numero_explorado = 0

        while True:


            if frontera.esta_vacia():
                raise Exception("No hay solución")
            nodo_actual = frontera.quitar_nodo()
            #print(nodo_actual.estado)
            #print(type(nodo_actual))
            if nodo_actual.estado == self.meta:
                print("Llegué a la meta")

                solucion=[]
                while nodo_actual.padre is not None:
                    solucion.append(nodo_actual.estado)
                    nodo_actual=nodo_actual.padre
                solucion.reverse()
                self.solucion = solucion
                print(self.solucion)
                self.dibujar(self.solucion)
                #TAREA: hacer la recapitulación de la solución recorriendo los padres de los nodos
                #ir guardando esos nodos en self.solución
                #devolver la secuencia de nodos recorridos
                return
            self.explorados.append(nodo_actual.estado) #Acá estaba el error... tenia que guardar los estados en vez de nodos
            self.numero_explorado+=1

            
            
          

            for posicion in self.expandir_nodo(nodo_actual):


                if (not frontera.contiene_estado(posicion) and not posicion in self.explorados ): #ahora si funciona ༼ つ ◕_◕ ༽つ
                   
                    hijo = Nodo(posicion,nodo_actual,self.meta,nodo_actual.costo_g + 1)
                    frontera.agregar_nodo(hijo)

               
        



In [144]:
import pygame
import sys

pygame.init()

WINDOW_SIZE = (800, 600)
screen = pygame.display.set_mode(WINDOW_SIZE)

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
red=(255,0,0)
gold=(255,215,0)
green=(0,128,0)
greenyellow=(173,255,47)
hotpink=(255,105,180)
lightcoral=(240,128,128)
maroon=(128,0,0)
orange=(255,165,0)
orangered=(255,69,0)



# Definir el texto a mostrar
text = "Seleccione el algortimo con el que desee resolver el Laberinto: "

# Definir la fuente y el tamaño del texto
font = pygame.font.SysFont("Arial", 25)

# Crear una superficie de texto
text_surface = font.render(text, True, (255,165,0))

# Crear una imagen vacía
image = pygame.Surface(WINDOW_SIZE)

# Pegar la superficie de texto en la imagen
image.blit(text_surface, (100, 100))

pygame.display.set_caption("|L|A|B|E|R|I|N|T|O")

# Crear los botones
button1_rect = pygame.Rect(100, 200, 150, 50)
button1_color = (255,165,0)
button1_text = "Depth First Search"
button1_font = pygame.font.SysFont("Arial", 18)
button1_text_surface = button1_font.render(button1_text, True, (0,0,0))

button2_rect = pygame.Rect(300, 200, 150, 50)
button2_color = (255,165,0)
button2_text = "Breadth First Search"
button2_font = pygame.font.SysFont("Arial", 18)
button2_text_surface = button2_font.render(button2_text, True, (0,0,0))

button3_rect = pygame.Rect(100, 400, 150, 50)
button3_color = (255,165,0)
button3_text = "Greedy First Search"
button3_font = pygame.font.SysFont("Arial", 18)
button3_text_surface = button3_font.render(button3_text, True, (0,0,0))

button4_rect = pygame.Rect(300, 400, 150, 50)
button4_color = (255,165,0)
button4_text = "A* Search"
button4_font = pygame.font.SysFont("Arial", 18)
button4_text_surface = button4_font.render(button4_text, True, (0,0,0))



# Mostrar los botones en la imagen
pygame.draw.rect(image, button1_color, button1_rect)
image.blit(button1_text_surface, (button1_rect.x + 10, button1_rect.y + 10))

pygame.draw.rect(image, button2_color, button2_rect)
image.blit(button2_text_surface, (button2_rect.x + 10, button2_rect.y + 10))


pygame.draw.rect(image, button3_color, button3_rect)
image.blit(button3_text_surface, (button3_rect.x + 10, button3_rect.y + 10))

pygame.draw.rect(image, button4_color, button4_rect)
image.blit(button4_text_surface, (button4_rect.x + 10, button4_rect.y + 10))

pygame.display.update()

# Esperar a que el usuario cierre la ventana
done= False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done= True
            pygame.quit()
            sys.exit()

        # Detectar clic en los botones
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = pygame.mouse.get_pos()
            if button1_rect.collidepoint(mouse_pos):
                laberinto=Laberinto('DFS')
                laberinto.resolver()
            elif button2_rect.collidepoint(mouse_pos):
                laberinto=Laberinto('BFS')
                laberinto.resolver()
            elif button3_rect.collidepoint(mouse_pos):
                laberinto=Laberinto('GBS')
                laberinto.resolver()
            elif button4_rect.collidepoint(mouse_pos):
                laberinto=Laberinto('GS')
                laberinto.resolver()
            
        #pygame.display.flip()
        screen.blit(image,(0, 0))
        
    # Actualizar la pantalla
    #pygame.quit()
    pygame.display.update()
    

Seleccione un archivo:
1. Laberinto_2.txt
2. Laberinto_3.txt
3. Laberinto_4.txt
Llegué a la meta
[(7, 7), (7, 6), (7, 5), (7, 4), (6, 4), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (4, 8), (3, 8), (2, 8), (1, 8), (1, 7), (1, 6), (2, 6), (3, 6), (3, 5), (3, 4), (3, 3)]


error: display Surface quit